# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch16/ch16-part1-self-attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 패키지 버전 체크

check_packages.py 스크립트에서 로드하기 위해 폴더를 추가합니다:

In [1]:
import sys

# 코랩의 경우 깃허브 저장소로부터 python_environment_check.py를 다운로드 합니다.
if 'google.colab' in sys.modules:
    !wget https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
else:
    sys.path.insert(0, '..')

--2023-09-05 09:11:39--  https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/python_environment_check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1629 (1.6K) [text/plain]
Saving to: ‘python_environment_check.py’

python_environment_ 100%[===================>]   1.59K  --.-KB/s    in 0s      

2023-09-05 09:11:41 (28.1 MB/s) - ‘python_environment_check.py’ saved [1629/1629]



권장 패키지 버전을 확인하세요:

In [2]:
from python_environment_check import check_packages


d = {
    'torch': '1.9.0',
}
check_packages(d)

[OK] Your Python version is 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
[OK] torch 2.0.1+cu118


# 16장 트랜스포머 - 어텐션 메커니즘을 통한 자연어 처리 성능 향상 (파트 1/3)

**목차**

- 어텐션 메커니즘이 추가된 RNN
  - RNN의 정보 검색을 돕는 어텐션
  - RNN을 위한 원본 어텐션 메커니즘
  - 양방향 RNN으로 입력 처리하기
  - 문맥 벡터에서 출력 생성하기
  - 어텐션 가중치 계산하기
- 셀프 어텐션 메커니즘 소개
  - 기본적인 형태의 셀프 어텐션
  - 훈련 가능한 셀프 어텐션 메카니즘: 스케일드 점곱 어텐션
- 어텐션이 필요한 전부다: 원본 트랜스포머 아키텍처
  - 멀티 헤드 어텐션으로 문맥 임베딩 인코딩하기
  - 언어 모델 학습: 디코더와 마스크드 멀티 헤드 어텐션
  - 구현 세부 사항: 위치 인코딩 및 층 정규화

In [3]:
from IPython.display import Image

## 어텐션 메커니즘이 추가된 RNN

### RNN의 정보 검색을 돕는 어텐션

In [4]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_01.png', width=500)

In [5]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_02.png', width=700)

### RNN을 위한 원본 어텐션 메커니즘

In [6]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_03.png', width=600)

### 양방향 RNN으로 입력 처리하기

### 문맥 벡터에서 출력 생성하기

### 어텐션 가중치 계산하기

## 셀프 어텐션 메커니즘 소개

### 기본적인 형태의 셀프 어텐션

- 딕셔너리로 인코딩된 입력 문장이 있다고 가정하고, RNN 장에서와 같이 단어를 정수로 매핑합니다:

In [7]:
import torch


# 입력 문장:
#  "Can you help me to translate this sentence"

sentence = torch.tensor(
    [0, # can
     7, # you
     1, # help
     2, # me
     5, # to
     6, # translate
     4, # this
     3] # sentence
)

sentence

tensor([0, 7, 1, 2, 5, 6, 4, 3])

- 다음으로 단어가 임베딩되어 있다고 가정합니다. 즉, 단어가 실수 벡터로 표현된다고 가정합니다.
- 단어가 8개이므로 벡터는 8개가 됩니다. 각 벡터는 16차원입니다:

In [8]:
torch.manual_seed(123)
embed = torch.nn.Embedding(10, 16)
embedded_sentence = embed(sentence).detach()
embedded_sentence.shape

torch.Size([8, 16])

- 문맥 벡터 $\boldsymbol{z}^{(i)}=\sum_{j=1}^{T} \alpha_{i j} \boldsymbol{x}^{(j)}$를 계산하는 것이 목표입니다. $\alpha_{i j}$는 어텐션 가중치입니다.
- 어텐션 가중치 $\alpha_{i j}$는 $\omega_{i j}$ 값을 사용합니다.
- 먼저 $\omega_{i j}$를 점곱으로 계산해 보죠:

$$\omega_{i j}=\boldsymbol{x}^{(i)^{\top}} \boldsymbol{x}^{(j)}$$

In [9]:
omega = torch.empty(8, 8)

for i, x_i in enumerate(embedded_sentence):
    for j, x_j in enumerate(embedded_sentence):
        omega[i, j] = torch.dot(x_i, x_j)

- 중첩된 for-루프를 행렬 곱셈으로 대체하여 더 효율적으로 계산해 보겠습니다:

In [10]:
omega_mat = embedded_sentence.matmul(embedded_sentence.T)

In [11]:
torch.allclose(omega_mat, omega)

True

- $\omega$를 정규화하여 어텐션 가중치를 계산하며 모두 더하면 1이 되도록 만듭니다.

$$\alpha_{i j}=\frac{\exp \left(\omega_{i j}\right)}{\sum_{j=1}^{T} \exp \left(\omega_{i j}\right)}=\operatorname{softmax}\left(\left[\omega_{i j}\right]_{j=1 \ldots T}\right)$$

$$\sum_{j=1}^{T} \alpha_{i j}=1$$

In [12]:
import torch.nn.functional as F

attention_weights = F.softmax(omega, dim=1)
attention_weights.shape

torch.Size([8, 8])

- 열의 합이 1이 되도록 할 수 있습니다:

In [13]:
attention_weights.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [14]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_04.png', width=700)

- 어텐션 가중치를 얻었으므로 어텐션 가중치 $\alpha_{i j}$를 사용하여 문맥 벡터 $\boldsymbol{z}^{(i)}=\sum_{j=1}^{T} \alpha_{i j} \boldsymbol{x}^{(j)}$를 계산합니다.
- 예를 들어 두 번째 입력 원소(인덱스가 1인 원소)의 문맥 벡터를 다음과 같이 계산할 수 있습니다:

In [15]:
x_2 = embedded_sentence[1, :]
context_vec_2 = torch.zeros(x_2.shape)
for j in range(8):
    x_j = embedded_sentence[j, :]
    context_vec_2 += attention_weights[1, j] * x_j

context_vec_2

tensor([-9.3975e-01, -4.6856e-01,  1.0311e+00, -2.8192e-01,  4.9373e-01,
        -1.2896e-02, -2.7327e-01, -7.6358e-01,  1.3958e+00, -9.9543e-01,
        -7.1287e-04,  1.2449e+00, -7.8077e-02,  1.2765e+00, -1.4589e+00,
        -2.1601e+00])

- 또는 더 효율적으로 선형 대수와 행렬 곱셈을 사용합니다:

In [16]:
context_vectors = torch.matmul(
        attention_weights, embedded_sentence)


torch.allclose(context_vec_2, context_vectors[1])

True

### 훈련 가능한 셀프 어텐션 메카니즘: 스케일드 점곱 어텐션

In [17]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_05.png', width=700)

In [18]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]
U_query = torch.rand(d, d)
U_key = torch.rand(d, d)
U_value = torch.rand(d, d)

In [19]:
x_2 = embedded_sentence[1]
query_2 = U_query.matmul(x_2)

In [20]:
key_2 = U_key.matmul(x_2)
value_2 = U_value.matmul(x_2)

In [21]:
keys = U_key.matmul(embedded_sentence.T).T
torch.allclose(key_2, keys[1])

True

In [22]:
values = U_value.matmul(embedded_sentence.T).T
torch.allclose(value_2, values[1])

True

In [23]:
omega_23 = query_2.dot(keys[2])
omega_23

tensor(14.3667)

In [24]:
omega_2 = query_2.matmul(keys.T)
omega_2

tensor([-25.1623,   9.3602,  14.3667,  32.1482,  53.8976,  46.6626,  -1.2131,
        -32.9392])

In [25]:
attention_weights_2 = F.softmax(omega_2 / d**0.5, dim=0)
attention_weights_2

tensor([2.2317e-09, 1.2499e-05, 4.3696e-05, 3.7242e-03, 8.5596e-01, 1.4026e-01,
        8.8897e-07, 3.1935e-10])

In [26]:
#context_vector_2nd = torch.zeros(values[1, :].shape)
#for j in range(8):
#    context_vector_2nd += attention_weights_2[j] * values[j, :]

#context_vector_2nd

In [27]:
context_vector_2 = attention_weights_2.matmul(values)
context_vector_2

tensor([-1.2226, -3.4387, -4.3928, -5.2125, -1.1249, -3.3041, -1.4316, -3.2765,
        -2.5114, -2.6105, -1.5793, -2.8433, -2.4142, -0.3998, -1.9917, -3.3499])

## 어텐션이 필요한 전부다: 원본 트랜스포머 아키텍처

In [28]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_06.png', width=600)

### 멀티 헤드 어텐션으로 문맥 임베딩 인코딩하기

In [29]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]
one_U_query = torch.rand(d, d)

In [30]:
h = 8
multihead_U_query = torch.rand(h, d, d)
multihead_U_key = torch.rand(h, d, d)
multihead_U_value = torch.rand(h, d, d)

In [31]:
multihead_query_2 = multihead_U_query.matmul(x_2)
multihead_query_2.shape

torch.Size([8, 16])

In [32]:
multihead_key_2 = multihead_U_key.matmul(x_2)
multihead_value_2 = multihead_U_value.matmul(x_2)

In [33]:
multihead_key_2[2]

tensor([-1.9619, -0.7701, -0.7280, -1.6840, -1.0801, -1.6778,  0.6763,  0.6547,
         1.4445, -2.7016, -1.1364, -1.1204, -2.4430, -0.5982, -0.8292, -1.4401])

In [34]:
stacked_inputs = embedded_sentence.T.repeat(8, 1, 1)
stacked_inputs.shape

torch.Size([8, 16, 8])

In [35]:
multihead_keys = torch.bmm(multihead_U_key, stacked_inputs)
multihead_keys.shape

torch.Size([8, 16, 8])

In [36]:
multihead_keys = multihead_keys.permute(0, 2, 1)
multihead_keys.shape

torch.Size([8, 8, 16])

In [37]:
multihead_keys[2, 1] # index: [2nd attention head, 2nd key]

tensor([-1.9619, -0.7701, -0.7280, -1.6840, -1.0801, -1.6778,  0.6763,  0.6547,
         1.4445, -2.7016, -1.1364, -1.1204, -2.4430, -0.5982, -0.8292, -1.4401])

In [38]:
multihead_values = torch.matmul(multihead_U_value, stacked_inputs)
multihead_values = multihead_values.permute(0, 2, 1)

In [39]:
multihead_z_2 = torch.rand(8, 16)

In [40]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_07.png', width=700)

In [41]:
linear = torch.nn.Linear(8*16, 16)
context_vector_2 = linear(multihead_z_2.flatten())
context_vector_2.shape

torch.Size([16])

### 언어 모델 학습: 디코더와 마스크드 멀티 헤드 어텐션

In [42]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_08.png', width=600)

### 구현 세부 사항: 위치 인코딩 및 층 정규화

In [43]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch16/figures/16_09.png', width=600)